In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
dataset = pd.read_csv('dataset.csv', index_col=False)
not_features = ["Path", "Emotional_intensity", "Statement", "Gender"]
features_data = dataset.drop(columns=not_features)

features_data.dropna(inplace=True)

In [3]:
#Elaborazione dei dati
actor_list = [1, 12, 7, 24]
test_set = features_data.loc[features_data['Actor'].isin(actor_list)]
training_set = features_data.drop(test_set.index)
training_actors = training_set['Actor'].unique()
training_actors_m = []
training_actors_f = []
for actor in training_actors:
    if (actor%2 == 0):
        training_actors_f.append(actor)
    else:
        training_actors_m.append(actor)

In [4]:
y_train = training_set["Emotions"]
X2d_train = training_set[dataset.columns[6:]]
y_test = test_set["Emotions"]
X2d_test = test_set[dataset.columns[6:]]


In [5]:
 # Define the random forest model
rfm = RandomForestClassifier(max_samples=0.9, n_estimators=200, max_depth=None, n_jobs=-1, random_state=42)
 # Define the grid
grid = {
    "max_samples": [0.3, 0.6, 0.7, 0.9, 0.11],
    "n_estimators": [200, 400, 600, 700, 800]
}
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
def cross_validation(model):
    parameters = {'max_samples': [0.5, 0.7, 0.9], 'n_estimators': [50, 100, 150]}
    best_accuracy = 0.0
    best_params = {}

    for max_samples in parameters['max_samples']:
        for n_estimators in parameters['n_estimators']:
            model_accuracy = []
            model_f1 = []

            for i in range(10):
                m = training_actors_m[i]
                f = training_actors_f[i]

                cross_test_set = training_set.loc[training_set['Actor'].isin([m, f])]
                cross_training_set = training_set.drop(cross_test_set.index)

                y_train = cross_training_set["Emotions"]
                X2d_train = cross_training_set[dataset.columns[6:]]
                y_test = cross_test_set["Emotions"]
                X2d_test = cross_test_set[dataset.columns[6:]]

                # Fit the classifier
                model.max_samples = max_samples
                model.n_estimators = n_estimators
                model.fit(X2d_train, y_train)

                # Compute the score and record it
                accuracy = model.score(X2d_test, y_test)
                model_accuracy.append(accuracy)

                y_pred = model.predict(X2d_test)
                f1 = f1_score(y_test, y_pred, average="macro")
                model_f1.append(f1)

            avg_accuracy = sum(model_accuracy) / len(model_accuracy)
            avg_f1 = sum(model_f1) / len(model_f1)

            if avg_accuracy > best_accuracy:
                best_accuracy = avg_accuracy
                best_params = {'max_samples': max_samples, 'n_estimators': n_estimators}

    print("Best parameters:", best_params)
    print("Best accuracy:", best_accuracy)

    return best_params, best_accuracy



In [7]:
rfm = RandomForestClassifier()
results = cross_validation(rfm)
print(results)


Best parameters: {'max_samples': 0.9, 'n_estimators': 100}
Best accuracy: 0.395
({'max_samples': 0.9, 'n_estimators': 100}, 0.395)


In [8]:
f1_score(y_test, y_pred, average="macro")

NameError: name 'y_pred' is not defined

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test, y_pred))

In [18]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
pd.DataFrame(cm, index=best_model.classes_, columns=best_model
             
             
             
             .classes_)

,angry,calm,disgust,fearful,happy,neutral,sad,surprised
angry,20,0,6,1,5,0,0,0
calm,0,29,0,0,0,0,3,0
disgust,0,4,21,1,3,1,2,0
fearful,3,0,2,14,4,0,7,2
happy,1,1,6,2,14,0,8,0
neutral,0,7,1,0,0,1,7,0
sad,0,6,8,4,0,0,14,0
surprised,0,0,3,2,4,0,4,19
